In [45]:
import json
from transformers import BertTokenizerFast
import uuid

In [46]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [56]:
label_map = {
    'B-1': 0,
    'I-1': 1,
    'B-2': 2,
    'I-2': 3,
    'B-3': 4,
    'I-3': 5,
    'B-4': 6,
    'I-4': 7,
    'B-5': 8,
    'I-5': 9,
    'B-6': 10,
    'I-6': 11,
}

result = {'label_map': label_map, 'examples': []}
with open('data/hierarchy.json') as rf:
    for ind, l in enumerate(rf.readlines()):
        l = l.strip()
        obj = json.loads(l)
        text = obj['text']
        labels = []
        curr_end = 0
        sorted_annotations = sorted(obj['annotations'], key=lambda x: x['start_offset'])
        for annotation in sorted_annotations:
            label = annotation['label']
            start_offset = annotation['start_offset']
            end_offset = annotation['end_offset']
            if end_offset <= curr_end:
                print(obj['annotations'])
                raise Exception('Something is wrong with the ordering of the offsets')
            curr_end = end_offset
            tokens = tokenizer.encode(text[start_offset:end_offset])
            seq_tokens = tokens[1:-1]
            for i in range(len(seq_tokens)):
                if i == 0:
                    tok_label = f'B-{label}'
                    labels.append(tok_label)
                else:
                    tok_label = f'I-{label}'
                    labels.append(tok_label)
        tokens = tokenizer.encode(text)
        if len(tokens)-2 != len(labels):
            print(ind)
        assert len(tokens)-2 == len(labels)
        if len(tokens) >= 513:
            # split it, stride 64
            t = tokens[1:-1]
            offset = 0
            ts = []
            ls = []
            while offset + 510 < len(t):
                toks = t[offset:offset+510]
                smalltext = tokenizer.decode(toks)
                ts.append(smalltext)
                lab = labels[offset:offset+510]
                if lab[0].startswith('I'):
                    lab[0] = 'B' + lab[0][1:]
                ls.append(lab)
                offset += 64
            toks = t[-510:]
            
            smalltext = tokenizer.decode(toks)
            ts.append(smalltext)
            lab = labels[-510:]
            assert len(toks) == len(lab)
            if lab[0].startswith('I'):
                lab[0] = 'B' + lab[0][1:]
            ls.append(lab)
            for t, l in zip(ts, ls):
                result['examples'].append({'original_ind': ind, 'text': t, 'labels': l})
        else:
            result['examples'].append({'original_ind': ind, 'text': text, 'labels': labels})
print(f'Number of examples: {len(result["examples"])}')
with open('data/labels.json', 'w') as wf:
    json.dump(result, wf)

Number of examples: 943


In [49]:
x = "A. Part I - Liability Coverage 1. The following is added to C. Exclusions: This coverage does not apply to emotional or mental distress, mental anguish, mental injury, or any similar injury unless it arises out of actual physical injury harm to that same person. 2. The following section is added: H. Service of Process If the insured’s whereabouts for service of process cannot be determined through reasonable effort, the  insured agrees to designate and irrevocably appoint  us as the agent of the  insured for service of process, pleadings, or other filings in a civil action brought against the  insured or to which the  insured has been joined as a defendant or respondent in any Colorado court if the cause of action concerns an incident for which the insured can possibly claim coverage. Subsequent termination of the insurance policy does not affect the appointment for an incident that occurred when the policy was in effect. The insured agrees that any such civil action may be commenced against the  insured by the service of process upon  us as if personal service had been made directly on the insured.  We agree to forward all communications related to service of process to the last-known e-mail and mailing address of the Named Insured in order to coordinate any payment of claims or defense of claims that are required. B. Cancellation or Nonrenewal 1. Cancellation by the Named Insured The Named Insured shown in the Declarations may cancel this policy by returning it to  us or by advising  us on what future date the cancellation is to be effective. 2. Cancellation by Us a. Cancellation of Policies in Effect for Less Than 60 Days We may cancel this policy for any reason by mailing notice of cancellation to the Named Insured at the address shown in the  Declarations not less than 10 days prior to the effective date of cancellation. b. Cancellation of Policies in Effect for 60 Days or More If this policy has been in effect for 60 days or more, or is a renewal or continuation policy, we will only cancel for the following reasons: (1) We will mail notice of cancellation to the Named Insured at the address shown in the  Declarations not less"
print(len(x))

2169


In [50]:
print(x[2057:2169])

 (1) We will mail notice of cancellation to the Named Insured at the address shown in the  Declarations not less


In [52]:
x = "A. Notify Us You or any person claiming coverage under this policy must promptly tell us all known details about the claim including but not limited to: 1. the time and place; and 2. the name and address of each: a. injured person; and b. witness. B. Other Duties 1. Each person claiming any coverage of this policy must: a. assist and cooperate with us in any matter concerning a claim or suit. b. promptly send us any legal papers relating to any claim or suit."
print(len(x))

463
